In [21]:
from IPython.core.release import author
# imports
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import OWL, RDF, RDFS, XSD
from owlready2 import *


In [22]:
NAMESPACE = "https://jaron.sprute.com/uni/actionable-knowledge-representation/feinschmecker#"

g = Graph()

# Create prefix fs_kg with given namespace
# You can define your own namespaces by passing any IRI and bind to an abbrevation
productkg = Namespace(NAMESPACE)
#even though the namespaces are imported already, they need to be bound to variables so they can be used
g.bind("feinschmecker", productkg)
g.bind("owl", OWL)


recipe = URIRef(NAMESPACE + "Recipe")
time = URIRef(NAMESPACE + "Time")
source = URIRef(NAMESPACE + "Source")
author = URIRef(NAMESPACE + "Author")
ingredient = URIRef(NAMESPACE + "Ingredient")

requiresTime = URIRef(NAMESPACE + "RequiresTime")
hasIngredient = URIRef(NAMESPACE + "hasIngredient")


g.add((recipe, RDFS.subClassOf, OWL.Thing))
g.add((time, RDFS.subClassOf, OWL.Thing))
g.add((source, RDFS.subClassOf, OWL.Thing))
g.add((author, RDFS.subClassOf, OWL.Thing))
g.add((ingredient, RDFS.subClassOf, OWL.Thing))

g.add((requiresTime, RDF.type, RDF.Property))
g.add((hasIngredient, RDF.type, RDF.Property))

g.add((recipe, hasIngredient, ingredient))

g.print()

@prefix feinschmecker: <https://jaron.sprute.com/uni/actionable-knowledge-representation/feinschmecker#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

feinschmecker:Author rdfs:subClassOf owl:Thing .

feinschmecker:Recipe rdfs:subClassOf owl:Thing ;
    feinschmecker:hasIngredient feinschmecker:Ingredient .

feinschmecker:RequiresTime a rdf:Property .

feinschmecker:Source rdfs:subClassOf owl:Thing .

feinschmecker:Time rdfs:subClassOf owl:Thing .

feinschmecker:hasIngredient a rdf:Property .

feinschmecker:Ingredient rdfs:subClassOf owl:Thing .




In [26]:

onto = get_ontology(NAMESPACE)

def ThingFactory(name, argnames, BaseClass=Thing):
    return newclass = type(name, (BaseClass,), {"__init__": __init__})
    def __init__(self, **kwargs):
        a=2
        BaseClass.__init__(self)
    newclass = type(name, (BaseClass,),{"__init__": __init__})
    return newclass

hello = ThingFactory("Hello", None)
